In [4]:
from pymatgen.io import cif
import pymatgen.core as mg
import glob
from tqdm import tqdm
import pandas as pd

# read cif files and extract prototype information

In [ ]:
cif_file = cif.CifParser(cif_path)
cf = cif_file.as_dict()
cif_dict = cf[list(cf.keys())[0]]

In [2]:
cif_dir = '../0_retrieve_icsd_data/final_cifs_for_aflow'

prototype_list = []
collection_code_list = []
formula_list = []


for cif_path in tqdm(glob.glob(f'{cif_dir}/*')):

    cif_file = cif.CifParser(cif_path)
    cf = cif_file.as_dict()
    cif_dict = cf[list(cf.keys())[0]]

    try: prototype = cif_dict['_chemical_name_structure_type']
    except: prototype = 'ERROR'
    try: collection_code = int(cif_dict['_database_code_ICSD'])
    except: collection_code = -1
    try: formula = mg.Structure.from_file(cif_path).composition.reduced_formula
    except: formula = 'ERROR'

    formula_list += [formula]
    collection_code_list += [collection_code]
    prototype_list += [prototype]

  0%|          | 0/14679 [00:00<?, ?it/s]/Users/andrewlee/miniforge3/envs/mat-hack/lib/python3.11/site-packages/pymatgen/io/cif.py:1167: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
  0%|          | 19/14679 [00:00<04:55, 49.66it/s] /Users/andrewlee/miniforge3/envs/mat-hack/lib/python3.11/site-packages/pymatgen/io/cif.py:1022: UserWarning: Some occupancies ([1.0, 1.012]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.0
  warnings.warn(msg)
/Users/andrewlee/miniforge3/envs/mat-hack/lib/python3.11/site-packages/pymatgen/io/cif.py:1162: UserWarning: No structure parsed for 1 structure in CIF. Section of CIF file below.
  warnings.warn(f"No structure parsed for {i + 1} structure in CIF. Section of CIF file below.")
/Users/andrewlee/min

# write results to dataframe

In [15]:
results = pd.DataFrame(data = {'Collection Code': collection_code_list,
                                'Formula': formula_list,
                                'Prototype': prototype_list})
results.to_csv('cif_files_prototypes_list.csv')

# get some stats about errors 

In [11]:
print(f'There are {len(results)} entries.')

# filter out errors
non_error_results = results[(results['Collection Code'] != -1) & (results['Formula'] != 'ERROR') & (results['Prototype'] != 'ERROR')]

print(f'There are {len(non_error_results)} entries with no errors.')

There are 14679 entries.
There are 13918 entries with no errors.


# analysis

In [14]:
non_error_results['Prototype'].value_counts().head(10)

Laves(cub)#MgCu2     2580
Laves(2H)#MgZn2      1244
Fluorite#CaF2         868
AlB2                  746
Rutile#TiO2           671
Pyrite#FeS2(cP12)     387
CeCu2                 345
CuAl2                 317
ThSi2                 254
CdI2(hP3)             254
Name: Prototype, dtype: int64